In [20]:
import numpy as np
import nltk # type: ignore 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize # type: ignore
from nltk.corpus import stopwords # type: ignore
import string
import numpy as np
from preprocess import extract_features, build_freq, word_freq
from sklearn.preprocessing import MinMaxScaler


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\obinw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [87]:
file_path_positive = '../data/positive_tweets.txt'
file_path_negative = '../data/negative_tweets.txt'
import math
pos_freqs = build_freq(file_path_positive)  # Build frequency dictionary
# print("The psoitive frequency")
# print(pos_freqs)
neg_freqs = build_freq(file_path_negative)
# print("The negative frquency")
# print(neg_freqs)
word_f = word_freq(pos_freqs, neg_freqs, "../data/sample_sentiment_text.txt")
#print("The frequency of the sentences ares ", word_f)
matrix = extract_features(word_f)
scaler = MinMaxScaler()
matrix = scaler.fit_transform(matrix)

print(matrix)

[[0.         0.625      0.33333333]
 [0.         0.25       0.46666667]
 [0.         0.375      0.06666667]
 [0.         0.4375     0.        ]
 [0.         0.125      0.46666667]
 [0.         0.0625     0.13333333]
 [0.         0.125      0.13333333]
 [0.         0.0625     0.        ]
 [0.         0.1875     0.2       ]
 [0.         0.0625     0.06666667]
 [0.         0.1875     0.2       ]
 [0.         0.         0.        ]
 [0.         0.125      0.13333333]
 [0.         0.875      0.        ]
 [0.         0.125      0.33333333]
 [0.         0.         0.46666667]
 [0.         0.9375     0.66666667]
 [0.         0.         0.26666667]
 [0.         0.5625     0.13333333]
 [0.         0.25       0.93333333]
 [0.         0.875      0.2       ]
 [0.         0.5625     1.        ]
 [0.         0.875      0.        ]
 [0.         0.5        0.93333333]
 [0.         0.375      0.        ]
 [0.         0.1875     0.66666667]
 [0.         0.625      0.6       ]
 [0.         0.25       0.66

In [128]:
import pandas as pd
rate = 0.001
#initialize some parameters
parameter = np.array([0.005, 0.0001, 0.0001])
b = 0.05
df = pd.read_csv("../data/output.txt")
m = len(df)
target = df.values
print(target)

[[1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]]


In [129]:
#I am building my classification model
#the logixtic model
def model(matrix, parameter, b):
    y_hat = []
    for row in matrix:
        feature = row
        z = np.dot(feature, parameter) + b
        g = 1/(1+(math.e)**(z))
        y_hat.append(g)
    return y_hat
print(y_hat)

[0.45239463635418925, 0.45983671857787534, 0.46978685356447, 0.46978685356447, 0.46480828456469886, 0.4797610644281854, 0.4797610644281854, 0.4847547270119201, 0.4722784572508355, 0.48225745272227183, 0.4722784572508355, 0.4872527628438865, 0.47726568640604844, 0.4573538673545378, 0.46978685356447, 0.46978685356447]


In [130]:
#getting my cost and loss functions
import math
#compute the loss function
def loss_function(target, y_hat):
    cost = 0
    for x in range(m):
        output = target[x,0]
        loss = (-1 *output * math.log(y_hat[x])) - (1-output) * math.log(1-y_hat[x])
        cost +=loss
    cost =  cost/m
    return cost

In [131]:
def gradient_descent(matrix, target, parameter, b, rate, iterations):
    print("Beginning Parameters:", parameter)
    print("beginning Bias:", b)
    m = len(target)
    for _ in range(iterations):
        y_hat = model(matrix, parameter, b)
        d_parameter = np.zeros_like(parameter)
        d_b = 0
        for i in range(m):
            d_parameter += (target[i] - y_hat[i]) * matrix[i]
            d_b += (target[i] - y_hat[i])
        parameter -= (rate / m) * d_parameter
        b -= (rate / m) * d_b
        
        if _ % 100 == 0:
            cost = loss_function(target, y_hat)
            print(f"Iteration {_}, Cost: {cost}")
    
    return parameter, b

iterations = 5000
parameter, b = gradient_descent(matrix, target, parameter, b, rate, iterations)
print("Updated Parameters:", parameter)
print("Updated Bias:", b)


Iteration 0, Cost: 0.6962404265671059
Iteration 100, Cost: 0.6982257229180159


C:\Users\obinw\AppData\Local\Temp\ipykernel_96060\165366476.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  loss = (-1 *output * math.log(y_hat[x])) - (1-output) * math.log(1-y_hat[x])


Iteration 200, Cost: 0.7002528000923327
Iteration 300, Cost: 0.70232404167878
Iteration 400, Cost: 0.7044419853399949
Iteration 500, Cost: 0.7066093322819467
Iteration 600, Cost: 0.7088289572113871
Iteration 700, Cost: 0.7111039187917865
Iteration 800, Cost: 0.7134374706058464
Iteration 900, Cost: 0.7158330726297969
Updated Parameters: [ 0.005      0.1059939 -0.0668421]
Updated Bias: [0.12934097]


In [1]:
#test data
decision_boundary = 0.5
#Now the model has been trained, now it is time to test the model and see how accyrate it is. 
parameters = np.array([-0.45760497, -1.10344803, 1.04100168])
b = [-0.41260497]

def model(matrix, parameter, b):
    y_hat = []
    for row in matrix:
        feature = row
        z = np.dot(feature, parameter) + b
        g = 1/(1+(math.e)**(z))
        print(g)
        y_hat.append(g)
    return y_hat
word_f = word_freq(pos_freqs, neg_freqs, "../data/test_sample.txt")
#print("The frequency of the sentences are ", word_f)
matrix = extract_features(word_f)
print("The matrix of the functions are: ", matrix)
y_hat = model(matrix, parameter, b)
#print("The prediction of the statements are: ", y_hat)

NameError: name 'np' is not defined